In [ ]:
from src.base.forecasting.models import LogLogAUCLoss
from src.base.forecasting.models.time_series import TimeSeriesModelAutoRegressiveMLP

from src.projects.fagradalsfjall.common.dataset import load_train_data_numpy
from src.projects.fagradalsfjall.common.model_eval import ModelEvalResult
from src.projects.fagradalsfjall.common.model_repo import ModelRepo
from src.projects.fagradalsfjall.common.project_settings import (
    CV_HORIZON_SAMPLES,
    TS_CV_SPLITTER,
    TS_PRIMARY_METRIC,
)
from src.projects.fagradalsfjall.blog_6_nn.param_grids import params_grid_search

# 1. Grid search for loss=LogLogAUC

In [ ]:
# -------------------------------------------------------------------------
#  Load TRAINING set
# -------------------------------------------------------------------------

print("Loading training dataset...    ", end="")
x_train = load_train_data_numpy()
print("Done.")

In [ ]:
# -------------------------------------------------------------------------
#  MLP - Some test runs without CV
# -------------------------------------------------------------------------

# --- define model ----------------------------------------
loss = LogLogAUCLoss()
model_mlp = TimeSeriesModelAutoRegressiveMLP(p=64, n=192, loss=loss)

models = {"mlp-auc": model_mlp}

# --- configure param optimization ------------------------
param_grid = params_grid_search

# --- fit - cross-validation ------------------------------
# should complete in ±6h with 12 workers
model_mlp.cv.grid_search(
    x=x_train,
    param_grid=param_grid,
    metric=TS_PRIMARY_METRIC,
    ts_cv_splitter=TS_CV_SPLITTER,
    hor=CV_HORIZON_SAMPLES,
    retrain=True,
    n_jobs=6,
)

In [ ]:
# plot grid search results
# model_mlp.cv.results.plot_2d(x_param="n_epochs", y_param="lr_max").create(w=18, h=12)

# model_mlp.cv.results.plot_1d(param_names="lr_max").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="n_epochs").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="n_hidden_layers").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="layer_width").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="dropout").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="wd").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="n").create(w=18, h=12)

In [ ]:
# --- evaluate & save -------------------------------------
model_eval_results = ModelEvalResult.eval_many(models, compute_metrics=True)

ModelRepo.save_models(model_eval_results)